In [ ]:
require 'BOAST'

In [ ]:
a = BOAST::Int "a"

In [ ]:
b = BOAST::Int "b"

In [ ]:
BOAST::decl a, b

In [ ]:
p = BOAST::Procedure("test_proc", [a , b ] )

In [ ]:
BOAST::opn p

In [ ]:
BOAST::close p

In [ ]:
BOAST::lang = BOAST::C

In [ ]:
BOAST::opn p

In [ ]:
BOAST::close p

In [ ]:
a = BOAST::Real("a",:dir => :in)

In [ ]:
b = BOAST::Real("b",:dir => :out)

In [ ]:
p = BOAST::Procedure("test_proc", [a , b ] ) {
  BOAST::pr b === a + 2
  }

In [ ]:
BOAST::lang = BOAST::FORTRAN

In [ ]:
BOAST::pr p

In [ ]:
include BOAST

In [ ]:
n = Int("n" , :dir => :in)
a = Real("a", :dir => :in, :dim => [Dim(n)])
b = Real("b", :dir => :out, :dim => [Dim(n)])
p = Procedure("test_proc", [n, a, b]) {
  decl i = Int("i")
  pr For(i, 1, n) {
    pr b[i] === a[i] + 2
  }
}

In [ ]:
k = p.ckernel

In [ ]:
k.build

In [ ]:
set_verbose(true)

In [ ]:
k.build

In [ ]:
input = NArray.float(1024).random
output = NArray.float(1024)
k.run(input.length, input, output)
(output - input ).each { |val|
  raise "Error !" if (val -2).abs > 1e-15
}
stats = k.run(input.length, input, output)
puts " #{ stats[:duration]} s"

In [ ]:
def vector_add
  kernel = CKernel::new
  function_name = "vector_add"
  n = Int("n", {:dir => :in, :signed => false})
  a = Real("a", {:dir => :in, :dim => [Dim(0, n -1)]})
  b = Real("b", {:dir => :in , :dim => [Dim(0 ,n -1)] })
  c = Real("c" ,{:dir => :out, :dim => [Dim(0, n -1)]})
  i = Int("i", {:signed => false})
  pr p = Procedure(function_name, [n, a , b, c]) {
    decl i
    if [CL, CUDA].include?(get_lang) then
      pr i === get_global_id(0)
      pr c[i] === a[i] + b[i]
    else
      pr For(i ,0 ,n -1) {
      pr c[i] === a[i] + b[i]
    }
    end
  }
  kernel.procedure = p
  return kernel
end

In [ ]:
n = 1024*1024
a = NArray.float( n ).random!
b = NArray.float( n ).random!
c = NArray.float( n )
epsilon = 10e-15
c_ref = a + b
nil

In [ ]:
[:FORTRAN, :C, :CL].each {|l|
  set_lang(BOAST.const_get(l))
  puts "#{l}:"
  k = vector_add
  puts k.print
  c.random!
  k.run(n, a, b, c, :global_work_size => [n ,1 ,1], :local_work_size => [32 ,1 ,1])
  diff = (c_ref - c).abs
  diff.each {|elem|
    raise "Warning: residue too big: #{elem}" if elem > epsilon
  }
}
puts "Success !"